### Homework: Open-Source LLMs
___
In this homework, we will experiment more with Ollama.

### Q1. Running Ollama with Docker
___
Let's run ollama with Docker. We will need to execute the same command as in the lectures:

```
docker run -it \
    --rm \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

What's the version of ollama client?

To find out, enter the container and execute ollama with the -v flag.

0.1.47

### Q2. Downloading an LLM
____

We will download a smaller LLM - gemma:2b.

Again let's enter the container and pull the model.

```
ollama pull gemma:2b
```

In docker, it saved the results into /root/.ollama

We're interested in the metadata about this model. You can find it in models/manifests/registry.ollama.ai/library

What's the content of the file related to gemma?

```
{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"applicati
on/vnd.docker.container.image.v1+json","digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea9942
90","size":483},"layers":[{"mediaType":"application/vnd.ollama.image.model","digest":"sha256:c1864a5eb19305c40519da12c
c543519e48a0697ecd30e15d5ac228644957d12","size":1678447520},{"mediaType":"application/vnd.ollama.image.license","diges
t":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca","size":8433},{"mediaType":"application/vn
d.ollama.image.template","digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871","size":136
},{"mediaType":"application/vnd.ollama.image.params","digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73e
fb8c6b4e2cd0eea0","size":84}]}
```

### Q3. Running the LLM
---
Test the following prompt: "10 * 10".  What's the answer?

```
'Sure, here is the response:\n\n10 * 10<sup>end_of_turn</sup>\n\nThis expression calculates 10 multiplied by 10<sup>end_of_turn</sup>, where end_of_turn is the variable representing the end of the number to be represented in scientific notation.'
```

### Q4. Downloading the weights
----
We don't want to pull the weights every time we run a docker container.  Let's do it once and have them available every time we start a container.

First, we will need to change how we run the container.

Instead of mapping the /root/.ollama folder to a named volume, let's map it to a local directory:

```
mkdir ollama_files

docker run -it \
    --rm \
    -v ./ollama_files:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

Now pull the model:

```
docker exec -it ollama ollama pull gemma:2b 
```

What's the size of the ollama_files/models folder?

* 0.6G
* 1.2G
* 1.7G
* 2.2G

Hint: on linux, you can use du -h for that.

```
1.6G    ./ollama_files/models
```

1.7G is the closest

### Q5. Adding the weights
---
Let's now stop the container and add the weights to a new image

For that, let's create a Dockerfile:

```
FROM ollama/ollama

COPY ...
```

What do you put after COPY?

```
COPY ./ollama_files/ /root/.ollama
```

### Q6. Serving it
---
Let's build it:

```
docker build -t ollama-gemma2b .
```

And run it:

```
docker run -it --rm -p 11434:11434 ollama-gemma2b
```

We can connect to it using the OpenAI client

In [ ]:
#!pip install ollama

In [1]:
import ollama
import tiktoken
from openai import OpenAI

In [2]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [3]:
encoding = tiktoken.encoding_for_model("gpt-4o")

Let's test it with the following prompt:

In [4]:
prompt = "What's the formula for energy?"

In [5]:
len(encoding.encode(prompt))

6

In [6]:
response = ollama.generate(model='gemma:2b', prompt=prompt)
print(response['response'])

Sure. The formula for energy is:

**E = W**

where:

* **E** is energy in joules (J)
* **W** is work done in joules (J)


In [7]:
len(encoding.encode(response['response']))

40

Also, to make results reproducible, set the temperature parameter to 0:

In [8]:
response = ollama.generate(model='gemma:2b', prompt=prompt, options=dict(temperature=0))
print(response['response'])

Sure, here's the formula for energy:

**E = K + U**

Where:

* **E** is the energy in joules (J)
* **K** is the kinetic energy in joules (J)
* **U** is the potential energy in joules (J)

**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:

**K = 1/2 * m * v^2**

**Potential energy (U)** is the energy an object possesses when it is in a position or has a specific configuration. It is calculated as the product of an object's mass and the gravitational constant (g) multiplied by the height or distance of the object from a reference point.

**Gravitational potential energy (U)** is given by the formula:

**U = mgh**

Where:

* **m** is the mass of the object in kilograms (kg)
* **g** is the acceleration due to gravity in meters per second squared (m/s^2)
* **h** is the height or distance of the object in meters (m)

The formula for energy can be used to calculate 

In [9]:
len(encoding.encode(response['response']))

283

In [10]:
print(response['response'])

Sure, here's the formula for energy:

**E = K + U**

Where:

* **E** is the energy in joules (J)
* **K** is the kinetic energy in joules (J)
* **U** is the potential energy in joules (J)

**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:

**K = 1/2 * m * v^2**

**Potential energy (U)** is the energy an object possesses when it is in a position or has a specific configuration. It is calculated as the product of an object's mass and the gravitational constant (g) multiplied by the height or distance of the object from a reference point.

**Gravitational potential energy (U)** is given by the formula:

**U = mgh**

Where:

* **m** is the mass of the object in kilograms (kg)
* **g** is the acceleration due to gravity in meters per second squared (m/s^2)
* **h** is the height or distance of the object in meters (m)

The formula for energy can be used to calculate 

In [11]:
len(encoding.encode(response['response']))

283

In [13]:
response

{'model': 'gemma:2b',
 'created_at': '2024-06-29T20:38:45.141750116Z',
 'response': "Sure, here's the formula for energy:\n\n**E = K + U**\n\nWhere:\n\n* **E** is the energy in joules (J)\n* **K** is the kinetic energy in joules (J)\n* **U** is the potential energy in joules (J)\n\n**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:\n\n**K = 1/2 * m * v^2**\n\n**Potential energy (U)** is the energy an object possesses when it is in a position or has a specific configuration. It is calculated as the product of an object's mass and the gravitational constant (g) multiplied by the height or distance of the object from a reference point.\n\n**Gravitational potential energy (U)** is given by the formula:\n\n**U = mgh**\n\nWhere:\n\n* **m** is the mass of the object in kilograms (kg)\n* **g** is the acceleration due to gravity in meters per second squared (m/s^2)\n* 

In [17]:
response['eval_count']

304

How many completion tokens did you get in response?

* 304
* 604
* 904
* 1204

304 completion tokens in the response.